In [5]:
# 3계층의 신경망으로 MNIST 데이터를 학습하는 코드

import numpy
# 시그모이드 함수 expit() 사용을 위해 scipy.special 불러오기
import scipy.special
# 행렬을 시각화 하기 위한 라이브러리
import matplotlib.pyplot
# 시각화가 외부 윈도우가 아닌 현재의 노트북 내에서 보이도록 설정
%matplotlib inline

In [6]:
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):
        # 입력, 은닉, 출력 계층의 노드 개수 설정
        self.innodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        # 가중치 행령 wih와 who
        # 배열 내 가중치는 w_i_j로 표기, 노드 i에서 다음 계층의 노드 j로 연결됨을 의미
        # w11 w21
        # w12 w22 등
        self.wih = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes1, self.innodes))
        self.whh = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.hnodes2, self.hnodes1))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes2))
        
        # 학습률
        self.lr = learningrate
        
        # 활성화 함수로는 시그모이드 함수를 이용
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    
    # 신경망 학습시키기
    def train(self, input_list, targets_list):
        # 입력 리스트를 2차원의 행렬로 변환
        inputs = numpy.array(input_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # 은닉 계층1으로 들어오는 신호를 계산
        hidden_inputs1 = numpy.dot(self.wih, inputs)
        # 은닉 계층1에서 나가는 신호를 계산
        hidden_outputs1 = self.activation_function(hidden_inputs1)
        
        # 은닉 계층2으로 들어오는 신호를 계산
        hidden_inputs2 = numpy.dot(self.whh, hidden_outputs1)
        # 은닉 계층2에서 나가는 신호를 계산
        hidden_outputs2 = self.activation_function(hidden_inputs2)
        
        # 최종 출력 계층으로 들어오는 신호를 계산
        final_inputs = numpy.dot(self.who, hidden_outputs2)
        # 최종 출력 계층에서 나가는 신호를 계산
        final_outputs = self.activation_function(final_inputs)
        
        # 출력 계층의 오차는 (실제 값 - 계산 값)
        output_errors = targets - final_outputs
        
        # 은닉 계층의 오차는 가중치에 의해 나뉜 출력 계층의 오차들을 재조합해 계산
        hidden_errors1 = numpy.dot(self.who.T, output_errors)
        hidden_errors2 = numpy.dot(self.whh.T, hidden_errors1)
        
        # 은닉 계층2과 출력 계층 간의 가중치 업데이트
        self.who += self.lr * numpy.dot((output_errors*final_outputs*(1.0-final_outputs)), numpy.transpose(hidden_outputs2))
        
        # 은닉 계층1과 은닉 계층2 간의 가중치 업데이트
        self.whh += self.lr * numpy.dot((hidden_errors2*hidden_outputs2*(1.0-hidden_outputs2)), numpy.transpose(hidden_outputs1))
        
        # 입력 계층과 은닉 계층1 간의 가중치 업데이트
        self.wih += self.lr * numpy.dot((hidden_errors1*hidden_outputs1*(1.0-hidden_outputs1)), numpy.transpose(inputs))
        
        pass
    
    # 신경망에 질의하기
    def query(self, inputs_list):
        # 입력 리스트를 2차원 행렬로 변환
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # 은닉 계층1으로 들어오는 신호를 계산
        hidden_inputs1 = numpy.dot(self.wih, inputs)
        # 은닉 계층1에서 나가는 신호를 계산
        hidden_outputs1 = self.activation_function(hidden_inputs1)
        
        # 은닉 계층2으로 들어오는 신호를 계산
        hidden_inputs2 = numpy.dot(self.whh, hidden_outputs1)
        # 은닉 계층2에서 나가는 신호를 계산
        hidden_outputs2 = self.activation_function(hidden_inputs2)
        
        # 최종 출력 계층으로 들어오는 신호를 계산
        final_inputs = numpy.dot(self.who, hidden_outputs2)
        # 최종 출력 계층에서 들어오는 신호를 계산
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [7]:
# 입력, 은닉, 출력 노드의 수
input_nodes = 784
hidden_nodes1 = 200
hidden_nodes2 = 200
output_nodes = 10

# 학습률
learning_rate = 0.01

# 신경망의 인스턴스를 생성
n = neuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

In [17]:
# mnist 학습 데이터인 csv 파일 리스트로 불러오기
training_data_file = open('fashion-mnist_train.csv', 'r')
next(training_data_file)
training_data_list = training_data_file.readlines()
training_data_file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
# 신경망 학습시키기

# 주기(epoch)란 학습 데이터가 학습을 위해 사용되는 횟수를 의미
epoch = 5

for e in range(epoch):
    # 학습 데이터 모음 내의 모든 레코드 탐색
    for record in training_data_list:
        # 레코드를 쉼표에 의해 분리
        all_values = record.split(',')
        # 입력 값의 범위와 값 조정
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # 결과 값 생성 (실제 값인 0.99 외에는 모두 0.01)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0]은 이 레코드에 대한 결과 값
        targets[int(all_values[0])] = 0.99
        n.train(inputs,targets)
        pass
    pass

In [22]:
# mnist 테스트 데이터의 csv 파일을 리스트로 불러오기
test_data_file = open('fashion-mnist_test.csv', 'r')
next(test_data_file)
# test_data_list = test_data_file.readlines()
# test_data_file.close()

['0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,44,88,99,122,123,80,0,0,0,0,1,1,1,0,0,0,0,0,0,1,2,0,0,0,3,46,174,249,67,0,94,210,61,14,212,157,37,0,0,0,0,1,0,0,0,0,0,2,2,0,23,168,206,242,239,238,214,125,61,113,74,133,236,238,236,203,184,20,0,1,0,0,0,0,0,1,0,0,175,245,223,207,205,206,216,255,237,251,232,223,212,200,205,216,249,173,0,0,2,0,0,0,0,7,0,53,225,201,197,200,201,206,199,197,185,194,204,232,226,249,219,194,205,229,33,0,1,0,0,0,0,1,0,133,223,208,192,195,233,226,216,191,210,188,236,186,0,50,234,207,208,231,133,0,0,0,0,0,0,0,0,216,218,216,194,229,172,64,219,201,200,200,247,68,72,54,165,237,212,219,226,0,0,0,0,0,0,0,50,221,207,220,211,207,165,138,205,192,191,190,232,119,113,67,173,237,217,208,221,29,0,0,0,0,0,0,131,216,200,219,207,212,231,226,193,214,224,206,203,230,122,112,234,224,214,204,224,123,0,0,0,0,0,0,195,212,204,211,203,205,200,184,213,162,138,193,207,203,231,245,208,220,211,203,219,179,0,0,0,0,0,8,185,191,218,233,219,201,221,213,246,11

In [14]:
# 신경망 테스트하기

# 신경망의 성능의 지표가 되는 성적표를 아무 값도 가지지 않도록 초기화
scorecard = []

# 테스트 데이터 모음 내에서 모든 레코드 탐색
for record in test_data_list:
    # 레코드를 쉼표에 의해 분리
    all_values = record.split(',')
    # 정답은 첫 번째 값
    correct_label = int(all_values[0])
    # 입력 값의 범위와 값 조정
    inputs = (numpy.asfarray(all_values[1:])/ 255.0 * 0.99) + 0.01
    # 신경망에 질의
    outputs = n.query(inputs)
    # 가장 높은 값의 인덱스는 레이블의 인덱스와 일치
    label = numpy.argmax(outputs)
    # 정답 또는 오답을 리스트에 추가
    if(label == correct_label):
        # 정답인 경우 성적표에 1을 더함
        scorecard.append(1)
    else:
        # 정답이 아닌 경우 성적표에 0을 더함
        scorecard.append(0)
        pass
    pass

In [15]:
# 정답의 비율인 성적을 계산해 출력
scorecard_array = numpy.asarray(scorecard)
print(f'epochs = {epoch}')
print(f'hidden_nodes1 = {hidden_nodes1}')
print(f'hidden_nodes2 = {hidden_nodes2}')
print(f'learning_rate = {learning_rate}')
print(f'performance = {scorecard_array.sum()/scorecard_array.size}')
print('201700949 설재혁')

epochs = 5
hidden_nodes1 = 200
hidden_nodes2 = 200
learning_rate = 0.01
performance = 0.8637
201700949 설재혁
